In [322]:
import textacy
import logging
import pandas as pd
import itertools
import math
from collections import Counter

LOGGER = logging.getLogger(__name__)

def make_stats_from_sents(doc):
    
    a = [Counter(map(lambda w_t: w_t[1], tagged_sen)) for tagged_sen in doc.pos_tagged_text]
    b = pd.DataFrame(a).fillna(0).agg(['mean', 'std'])
    c = b.unstack()
    n_labels = c.index.map(lambda l: 'sents_{}_{}'.format(*l))
    c.index = n_labels
    
    return c

# TODO: use hierarchical multiindex to group related metrics, like basic metrics and pos metrics
def make_stats(text, name):
    doc = textacy.Doc(text, lang="pt")

    stats = textacy.text_stats.TextStats(doc)

    # POS counter for all the text
    c = Counter([tag for (w, tag) in itertools.chain.from_iterable(doc.pos_tagged_text)])
    
    df = pd.Series(dict(stats.basic_counts, **c))
    
    # POS counter statistics for sentences
    ss = make_stats_from_sents(doc)
    df = pd.concat([df, ss])
    
    # Readability Metrics
    df['flesch_reading_ease'] = (-84.6 * stats.n_syllables / stats.n_words) - (1.015 * stats.n_words / stats.n_sents) + 206.835
    df['flesch_kincaid_grade_leve'] = (11.8 * stats.n_syllables / stats.n_words) + (0.39 * stats.n_words / stats.n_sents) - 15.59
    if stats.n_sents < 30:
        LOGGER.warning('SMOG score may be unreliable for n_sents < 30')
    df['smog_index'] = (1.0430 * math.sqrt(30 * stats.n_polysyllable_words / stats.n_sents)) + 3.1291
    df['gunning_fog_index'] = 0.4 * ((stats.n_words / stats.n_sents) + (100 * stats.n_polysyllable_words / stats.n_words))
    df['coleman_liau_index'] = (5.879851 * stats.n_chars / stats.n_words) - (29.587280 * stats.n_sents / stats.n_words) - 15.800804
    df['automated_readability_index'] = (4.71 * stats.n_chars / stats.n_words) + (0.5 * stats.n_words / stats.n_sents) - 21.43
    df['lix'] = (stats.n_words / stats.n_sents) + (100 * stats.n_long_words / stats.n_words)
    df['gulpease_index'] = (300 * stats.n_sents / stats.n_words) - (10 * stats.n_chars / stats.n_words) + 89
    # TODO: wiener_sachtextformel
    
    df.name = name
    
    return df.fillna(0)

In [305]:
# source: https://www2.uol.com.br/sciam/noticias/cientistas_implantam_cerebros_humanos_em_miniatura_em_cranios_de_camundongos.html
s1 = """Mas a verossimilhança dos organoides cerebrais humanos é limitada: quando crescem mais do que alguns milímetros, o oxigênio e os nutrientes não chegam às células mais internas. “Em nossas mãos, os organoides pararam de crescer após mais ou menos cinco semanas” diz Fred Gage, que liderou o estudo. “É um problema ligado ao tamanho,  não à idade. Algumas células morrem mesmo no pico de criação do organoide, a partir da décima semana. E isso piora com o tempo."""
# source: http://chc.org.br/pequenos-notaveis/
s2 = """Esses peixes são praticamente invisíveis aos nossos olhos por serem bastante pequenos, com poucos centímetros de comprimento. Por isso, encontrar um deles em um recife de coral é praticamente achar uma agulha em um palheiro! Com um tamanho tão pequeno e vivendo em um ambiente repleto de predadores, a maioria dos peixes criptobênticos vive em esconderijos: em pequenas cavidades dos recifes, dentro de esponjas, entre os ramos de lírios e ouriços-do-mar, corais, anêmonas…"""

In [306]:
len(s1), len(s2)

(460, 473)

In [309]:
pd.concat([make_stats(s1, 'scientific'), make_stats(s2, 'child')], axis=1).fillna(0)

SMOG score may be unreliable for n_sents < 30
SMOG score may be unreliable for n_sents < 30


,scientific,child
ADJ,4.000000,8.000000
ADJ_mean,0.666667,2.000000
ADJ_std,0.816497,1.414214
ADP,13.000000,18.000000
ADP_mean,2.166667,4.500000
ADP_std,1.471960,0.577350
ADV,7.000000,5.000000
ADV_mean,1.166667,1.250000
ADV_std,1.940790,0.500000
AUX,1.000000,2.000000


# Clarice x Machado

In [310]:
import codecs 

with codecs.open('../data/perto_do_coracao_selvagem.txt', 'r', 'utf-8') as f:
    clarice = f.read()
    
with codecs.open('../data/dom_casmurro.txt', 'r', 'utf-8') as f:
    machado = f.read()

In [311]:
len(clarice), len(machado)

(291114, 375295)

In [323]:
df = pd.concat([make_stats(clarice, 'Clarice Lispector'), make_stats(machado, 'Machado de Assis')], axis=1)
df

,Clarice Lispector,Machado de Assis
ADJ,3255.000000,3183.000000
ADP,5790.000000,7375.000000
ADV,4888.000000,5961.000000
AUX,670.000000,1128.000000
CCONJ,1586.000000,2779.000000
DET,6581.000000,9113.000000
INTJ,68.000000,68.000000
NOUN,9288.000000,12048.000000
NUM,188.000000,493.000000
PART,2.000000,NaN


In [331]:
df_sent = df.loc[df.index[df.index.str.startswith('sent')].tolist(), :]


    ADJ: adjective
    ADP: adposition
    ADV: adverb
    AUX: auxiliary verb
    CONJ: coordinating conjunction
    DET: determiner
    INTJ: interjection
    NOUN: noun
    NUM: numeral
    PART: particle
    PRON: pronoun
    PROPN: proper noun
    PUNCT: punctuation
    SCONJ: subordinating conjunction
    SYM: symbol
    VERB: verb
    X: other


In [335]:
(df_sent['Clarice Lispector'] - df_sent['Machado de Assis'])\
.sort_values(ascending=False)\
.apply(lambda v: '{:.2f} %'.format(v*100))

sents_SPACE_mean     51.76 %
sents_SPACE_std      31.68 %
sents_ADJ_mean       18.79 %
sents_ADJ_std        18.18 %
sents_ADV_mean        8.15 %
sents_ADV_std         6.69 %
sents_ADP_mean        4.33 %
sents_X_std           4.01 %
sents_NOUN_mean       3.25 %
sents_INTJ_std        2.27 %
sents_X_mean          1.87 %
sents_INTJ_mean       0.37 %
sents_NOUN_std       -0.05 %
sents_ADP_std        -1.31 %
sents_PRON_std       -2.00 %
sents_SCONJ_std      -3.51 %
sents_SYM_mean       -3.80 %
sents_AUX_mean       -4.16 %
sents_NUM_mean       -4.16 %
sents_SCONJ_mean     -4.88 %
sents_AUX_std        -5.06 %
sents_PUNCT_mean     -5.75 %
sents_PRON_mean      -5.86 %
sents_VERB_mean      -6.05 %
sents_DET_mean       -7.49 %
sents_VERB_std       -7.61 %
sents_SYM_std        -8.07 %
sents_NUM_std       -11.29 %
sents_CCONJ_mean    -11.70 %
sents_CCONJ_std     -13.28 %
sents_DET_std       -15.41 %
sents_PUNCT_std     -20.47 %
sents_PROPN_mean    -28.68 %
sents_PROPN_std     -38.92 %
sents_PART_mea